In [79]:
import h5py
import numpy as np
import os

In [80]:
root = os.path.abspath("..")
data_root = os.path.join(root, "datasets", "libero_10")
filename = "KITCHEN_SCENE3_turn_on_the_stove_and_put_the_moka_pot_on_it_demo.hdf5"
data = h5py.File(os.path.join(data_root, filename), "r")

In [82]:
# loop over tasks
for file in os.listdir(data_root):
    # print task name
    print(f"Eval Task: {file}")
    data = h5py.File(os.path.join(data_root, filename), "r")['data']

    # loop over demos
    # if this ends up being slow we can just use some percentage of the demos
    total_mse = 0
    for demo in data.keys():
        
        traj = data[demo]

        # save actions
        true_actions = np.asarray(traj['actions'])

        # get predicted actions
        pred_actions = []
        for obs in traj['obs']['agentview_rgb']:
            pred_actions.append(np.zeros(7))  # TODO: get actions predicted by diffusion policy
        pred_actions = np.stack(pred_actions)
        
        total_mse += np.mean(np.power(true_actions - pred_actions, 2))
    total_mse /= len(data.keys())
    print(f"\tAction MSE: {total_mse}")


Eval Task: KITCHEN_SCENE3_turn_on_the_stove_and_put_the_moka_pot_on_it_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: KITCHEN_SCENE4_put_the_black_bowl_in_the_bottom_drawer_of_the_cabinet_and_close_it_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: KITCHEN_SCENE6_put_the_yellow_and_white_mug_in_the_microwave_and_close_it_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: KITCHEN_SCENE8_put_both_moka_pots_on_the_stove_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: LIVING_ROOM_SCENE1_put_both_the_alphabet_soup_and_the_cream_cheese_box_in_the_basket_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: LIVING_ROOM_SCENE2_put_both_the_alphabet_soup_and_the_tomato_sauce_in_the_basket_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: LIVING_ROOM_SCENE2_put_both_the_cream_cheese_box_and_the_butter_in_the_basket_demo.hdf5
	Action MSE: 0.19106465670767583
Eval Task: LIVING_ROOM_SCENE5_put_the_white_mug_on_the_left_plate_and_put_the_yellow_and_white_mug_on_the_right_pl